<a href="https://colab.research.google.com/github/leti-olabarri/premier-fantasy/blob/feature%2Fload_data/models/Basic_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing

import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

from sklearn.model_selection import train_test_split

In [7]:
!wget /resources/data/Vir.txt https://drive.google.com/file/d/1YUKdlBqhPnNW0h40bFVlDusgaAbESUrX/

/resources/data/Vir.txt: Scheme missing.
--2021-10-22 16:07:25--  https://drive.google.com/file/d/1YUKdlBqhPnNW0h40bFVlDusgaAbESUrX/
Resolving drive.google.com (drive.google.com)... 172.253.114.102, 172.253.114.138, 172.253.114.100, ...
Connecting to drive.google.com (drive.google.com)|172.253.114.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/file/d/1YUKdlBqhPnNW0h40bFVlDusgaAbESUrX/edit [following]
--2021-10-22 16:07:25--  https://drive.google.com/file/d/1YUKdlBqhPnNW0h40bFVlDusgaAbESUrX/edit
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [ <=>                ]  65.15K  --.-KB/s    in 0.001s  

2021-10-22 16:07:25 (78.8 MB/s) - ‘index.html’ saved [66711]

FINISHED --2021-10-22 16:07:25--
Total wall clock time: 0.6s
Downloaded: 1 files, 65K in 0.001s (78.8 MB/s)


In [12]:
def startsWithDateAndTime(s):
    pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]) ([0-9]+):([0-9][0-9]) -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

def getDataPoint(line):   
    splitLine = line.split(' - ')
    dateTime = splitLine[0]
    date, time = dateTime.split(' ') 
    message = ' '.join(splitLine[1:])
    splitMessage = message.split(': ')
    author = splitMessage[0]
    message = ' '.join(splitMessage[1:])
    return date, time, author, message

parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = '/content/Chat de WhatsApp con Vir.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline() 
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip()
        if startsWithDateAndTime(line):
            if len(messageBuffer) > 0: 
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear()
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

In [13]:
df.head()

,Date,Time,Author,Message
0,2019-12-16,20:28,Vir,Mañana nos juntamos media horita y vemos un po...
1,2019-12-16,20:28,Vir,?!
2,2019-12-16,20:28,Vir,Un poco a modo de recuperar las weekly que ten...
3,2019-12-16,21:10,Leticia Olábarri,"Te los agradecería muchísimo, la verdad"
4,2019-12-16,21:11,Leticia Olábarri,*lo


In [14]:
def transform_names(name):
    if name == "Leticia Olábarri":
        return 1
    else:
        return 0

df["Author"] = df["Author"].apply(transform_names)

In [44]:
df.head()

,Date,Time,Author,Message
0,2019-12-16,20:28,0,Mañana nos juntamos media horita y vemos un po...
1,2019-12-16,20:28,0,?!
2,2019-12-16,20:28,0,Un poco a modo de recuperar las weekly que ten...
3,2019-12-16,21:10,1,"Te los agradecería muchísimo, la verdad"
4,2019-12-16,21:11,1,*lo


In [45]:
df.drop(df.tail(1).index,inplace=True)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(df["Message"], df["Author"], test_size=0.5, random_state=42)

In [70]:
labels = y_train.unique()
labels

array([1, 0])

In [64]:
max_features = 10000
sequence_length = 1000

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [65]:
vectorize_layer.adapt(X_train)

In [66]:
train_ds = vectorize_layer(X_train)
test_ds = vectorize_layer(X_test)

In [81]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_14 (Dropout)         (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_7 ( (None, 16)                0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [82]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [52]:
print(len(train_ds))
print(len(test_ds))

3545
3545


In [83]:
epochs = 10
history = model.fit(
    train_ds,y_train,
    validation_data= (test_ds, y_test),
    epochs=epochs)

Epoch 1/10
111/111 [==============================] - 2s 16ms/step - loss: 0.6934 - binary_accuracy: 0.5038 - val_loss: 0.6931 - val_binary_accuracy: 0.5007
Epoch 2/10
111/111 [==============================] - 2s 16ms/step - loss: 0.6933 - binary_accuracy: 0.4973 - val_loss: 0.6930 - val_binary_accuracy: 0.5007
Epoch 3/10
111/111 [==============================] - 2s 16ms/step - loss: 0.6931 - binary_accuracy: 0.5052 - val_loss: 0.6930 - val_binary_accuracy: 0.6350
Epoch 4/10
111/111 [==============================] - 2s 16ms/step - loss: 0.6932 - binary_accuracy: 0.4790 - val_loss: 0.6929 - val_binary_accuracy: 0.5035
Epoch 5/10
111/111 [==============================] - 2s 15ms/step - loss: 0.6928 - binary_accuracy: 0.5168 - val_loss: 0.6928 - val_binary_accuracy: 0.5007
Epoch 6/10
111/111 [==============================] - 2s 16ms/step - loss: 0.6931 - binary_accuracy: 0.5044 - val_loss: 0.6928 - val_binary_accuracy: 0.5927
Epoch 7/10
111/111 [==============================] - 2s 1